In [154]:
###DEBUG###
import os

os.environ['API_KEY'] = "889baad2-02b0-4c78-abe1-1952700d33ef" 
os.environ['NONINTERACT'] = 'True'

In [155]:
import json
import math
import os
import random
import sys
import time
import gc

In [156]:
import xarray as xr
import numpy as np
import pandas as pd
import random

import qnt.ta as qnta
import qnt.data as qndata
import qnt.backtester as qnbt

import qnt.stats as qnstats
import qnt.output as output
import qnt.graph as qngraph

In [157]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [158]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

class ElecDataset(Dataset):
    def __init__(self,feature,target):
        self.feature = feature
        self.target = target
    
    def __len__(self):
        return len(self.feature)
    
    def __getitem__(self,idx):
        item = self.feature[idx]
        label = self.target[idx]
        
        return item, label

In [159]:
def load_data():
    data = qndata.cryptofutures.load_data(#min_date = "2002-01-01", 
                                    max_date = "2013-12-31", 
                                    #tail=period,
                                   )
    return data

data = load_data()
close = data.sel(field="close").ffill('time').bfill('time')#.fillna(0)
asset_name_all = data.coords['asset'].values
print(list(asset_name_all))
print(len(asset_name_all))

['BTC']
1


In [160]:
for asset in asset_name_all:
    try:
        os.remove('generalcheckpoint.pt')
        os.remove('fractions.nc.gz')
        filename = asset+'checkpoint.pt'
        os.remove(filename)
    except: 
        pass

In [161]:
in_seq1 = []

for asset in asset_name_all:
    single_close = close.sel(asset=[asset]).to_pandas()
    print(single_close)
    close_ret = np.log(single_close/single_close.shift(1)).dropna()
    close_arr = np.array(close_ret)
    in_seq1.extend(close_arr)

asset            BTC
time                
2013-04-01  104.0100
2013-04-02  115.7625
2013-04-03  123.0000
2013-04-04  134.1200
2013-04-05  144.5000
...              ...
2013-12-27  711.0400
2013-12-28  721.1193
2013-12-29  728.8000
2013-12-30  736.4402
2013-12-31  728.0000

[275 rows x 1 columns]


In [162]:
#scaling 
from sklearn.preprocessing import MinMaxScaler
scaler_close = MinMaxScaler()
close_scaled = scaler_close.fit_transform(in_seq1)

In [163]:
# choose a number of time steps
n_steps = 100
# split into samples
X, y = split_sequence(close_scaled, n_steps)
y_reverse = scaler_close.inverse_transform(y)
y = np.where(y_reverse < 0, -1, 1 )

In [164]:
split = int(len(X)*0.8)
train_x = np.array(X)
valid_x = np.array(X[split:])
train_y = y
valid_y = y[split:]

train = ElecDataset(train_x.reshape(train_x.shape[0], 1, train_x.shape[1]),train_y)
valid = ElecDataset(valid_x.reshape(valid_x.shape[0], 1, valid_x.shape[1]),valid_y)
train_loader = torch.utils.data.DataLoader(train,batch_size=64,shuffle=True, drop_last = True)
print(len(train_loader))
valid_loader = torch.utils.data.DataLoader(valid,batch_size=64,shuffle=True, drop_last = True)
print(len(valid_loader))

2
0


CNN-1D Model

In [165]:
class TS(nn.Module):
    def __init__(self, n_input= 1, n_output=1):
        super(TS, self).__init__()

        self.conv1 = nn.Conv1d(1, 1, kernel_size = 3)
        self.conv2 = nn.Conv1d(1, 1, kernel_size = 3)
        self.pool1d = nn.MaxPool1d(2,2)
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU(inplace=True)
        self.fc1 = nn.Linear(23,4)
        self.fc2 = nn.Linear(4, n_output)
        #self.double()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool1d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool1d(x)
        # x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


model = TS(n_input=1, n_output=1)
model.to(device)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

n = count_parameters(model)
print("Number of parameters: %s" % n)

TS(
  (conv1): Conv1d(1, 1, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(1, 1, kernel_size=(3,), stride=(1,))
  (pool1d): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (relu): ReLU(inplace=True)
  (fc1): Linear(in_features=23, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=1, bias=True)
)
Number of parameters: 109


In [166]:
train_losses = []
valid_losses = []

def Train():
    
    running_loss = .0
    
    model.train()
    
    for idx, (inputs, labels) in enumerate(train_loader):
        labels = labels.unsqueeze(1)
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        preds = model(inputs.float())
        loss = criterion(preds,labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss
        
    train_loss = running_loss/len(train_loader)
    train_losses.append(train_loss.cpu().detach().numpy())
    
    print(f'train_loss {train_loss}')

def Valid():
    running_loss = .0
    
    model.eval()
    
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(valid_loader):
            labels = labels.unsqueeze(1)
            inputs = inputs.to(device)
            #print(inputs)
            labels = labels.to(device)
            #print(labels)
            optimizer.zero_grad()
            preds = model(inputs.float())
            #print(preds)
            loss = criterion(preds,labels)
            running_loss += loss
            
        valid_loss = running_loss/len(valid_loader)
        valid_losses.append(valid_loss.cpu().detach().numpy())
        print(f'valid_loss {valid_loss}')

In [167]:
epochs = 5
for epoch in range(epochs):
    print('epochs {}/{}'.format(epoch+1,epochs))
    Train()
    #Valid()
    gc.collect()

epochs 1/5
train_loss 0.8795068860054016
epochs 2/5
train_loss 0.891888439655304
epochs 3/5
train_loss 0.9126110076904297
epochs 4/5
train_loss 0.8770571947097778
epochs 5/5
train_loss 0.8752866387367249


In [168]:
def save_ckp(state, checkpoint_dir):
        f_path = checkpoint_dir
        torch.save(state, f_path)

checkpoint = {
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict()
            }
save_path = 'generalcheckpoint.pt'
save_ckp(checkpoint, save_path)

In [169]:
# https://quantiacs.com/documentation/en/examples/machine_learning_ridgeclassifier.html

import xarray as xr

import qnt.backtester as qnbt
import qnt.data as qndata
import numpy as np
import logging
import torch

def load_data(period):
    return qndata.cryptofutures.load_data(
        tail=period)

def predict_weights(market_data):
    
    def save_ckp(state, checkpoint_dir):
        f_path = checkpoint_dir
        torch.save(state, f_path)
  
    def load_ckp(checkpoint_fpath, model, optimizer):
        checkpoint = torch.load(checkpoint_fpath)
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        return model, optimizer, checkpoint['epoch']
        
    def define_model():
        model = TS(n_input=1, n_output=1)
        return model

    def Train():
        running_loss = .0
        model.train()
        
        for idx, (inputs, labels) in enumerate(train_loader):
            labels = labels.unsqueeze(1)
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            preds = model(inputs.float())
            loss = criterion(preds,labels.float())
            loss.backward()
            optimizer.step()
            running_loss += loss
            
        train_loss = running_loss/len(train_loader)
        train_losses.append(train_loss.cpu().detach().numpy())
        #print(f'train_loss {train_loss}')

    data = market_data.copy(True)
    close = data.sel(field="close").ffill('time').bfill('time').fillna(0).to_pandas()
    asset_name_all = data.coords['asset'].values
    predict_weights_next_day_df = data.sel(field="close").isel(time=-1).to_pandas()

    for asset_name in asset_name_all:
        close_returns = np.log(close[asset_name]/close[asset_name].shift(1)).dropna()
        close_array = np.array(close_returns)
        in_seq1 = close_array[:-1]
        
        #scaling
        close_scaled = scaler_close.transform(in_seq1.reshape(-1, 1))
        # choose a number of time steps
        n_steps = 100
        # split into samples
        X, y = split_sequence(close_scaled, n_steps)
        y_reverse = scaler_close.inverse_transform(y)
        y = np.where(y_reverse < 0, -1, 1 )

        # dataloader
        train = ElecDataset(X.reshape(X.shape[0],1, X.shape[1])[-5:],y[-5:])
        train_loader = torch.utils.data.DataLoader(train,batch_size=1,shuffle=True)
        
        # model and optimizer
        model = define_model()
        model.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
        criterion = nn.MSELoss()

        try:
            # load from 
            try:
                ckp_path = asset_name+'checkpoint.pt'
                model, optimizer, start_epoch = load_ckp(ckp_path, model, optimizer)
                #print("loading asset model")
            except: 
                ckp_path = 'generalcheckpoint.pt'
                model, optimizer, start_epoch = load_ckp(ckp_path, model, optimizer)
                #print("loading general model") 

            # train the model with current data
            epochs = 1
            train_losses = []
            for epoch in range(epochs):
                #print('epochs {}/{}'.format(epoch+1,epochs))
                Train()
                gc.collect()

            # save the model
            checkpoint = {
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict()
            }
            save_path = asset_name+'checkpoint.pt'
            save_ckp(checkpoint, save_path)
            
            x = close_scaled[-n_steps:]
            inputs = torch.from_numpy(x.reshape(1, 1, n_steps))
            
            model.eval()
            with torch.no_grad():
                inputs = inputs.to(device)
                optimizer.zero_grad()
                preds = model(inputs.float())
           
            predict_weights_next_day_df[asset_name] = preds[0]

        except:
            logging.exception("model failed")
            # if there is exception, return zero values
            return xr.zeros_like(data.isel(field=0, time=0))

    #print(predict_weights_next_day_df)
    return predict_weights_next_day_df.to_xarray()


weights = qnbt.backtest(
    competition_type="cryptofutures",
    load_data=load_data,
    lookback_period=200,
    start_date='2014-01-01',
    #test_period = 3,
    strategy=predict_weights,
    analyze=True,
    build_plots=True
)

Run last pass...
Load data...


| |#                                               | 5428 Elapsed Time: 0:00:00


Run strategy...
Load data for cleanup...
Output cleaning...
fix uniq
Normalization...
Output cleaning is complete.
Write result...


NOTICE: The environment variable OUTPUT_PATH was not specified. The default value is 'fractions.nc.gz'


Write output: fractions.nc.gz
---
Run first pass...
Load data...


...

---
Align...
Calc global stats...
---
Calc stats per asset...
Build plots...
---
Output:


asset,BTC
time,
2021-07-14,0.053411
2021-07-15,0.051929
2021-07-16,0.050197
2021-07-17,0.049100
2021-07-18,0.047666
2021-07-19,0.046492
2021-07-20,0.044573
2021-07-21,0.042609
2021-07-22,0.039647


Stats:


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2021-07-14,2.893123,0.000474,0.136542,-0.069715,-0.171003,1.107911,0.151276,1.0,1.0,0.007105,49.990381
2021-07-15,2.888257,-0.001682,0.136519,-0.071280,-0.171003,1.105782,0.150960,1.0,1.0,0.007103,50.006854
2021-07-16,2.888573,0.000110,0.136494,-0.071178,-0.171003,1.105674,0.150918,1.0,1.0,0.007100,50.045132
2021-07-17,2.887994,-0.000201,0.136469,-0.071364,-0.171003,1.105218,0.150828,1.0,1.0,0.007099,50.090122
2021-07-18,2.887626,-0.000127,0.136445,-0.071483,-0.171003,1.104844,0.150750,1.0,1.0,0.007096,50.118749
2021-07-19,2.884138,-0.001208,0.136421,-0.072604,-0.171003,1.103255,0.150507,1.0,1.0,0.007094,50.156400
2021-07-20,2.879322,-0.001670,0.136398,-0.074153,-0.171003,1.101144,0.150194,1.0,1.0,0.007092,50.187380
2021-07-21,2.888122,0.003056,0.136377,-0.071323,-0.171003,1.104293,0.150600,1.0,1.0,0.007089,50.238302
2021-07-22,2.890024,0.000659,0.136352,-0.070712,-0.171003,1.104798,0.150642,1.0,1.0,0.007089,50.290706


---


cryptofutures:
1. input_dim = 50 (1 epoch)
sr = 1.03
2. input_dim = 40 (1 epoch)
sr = 1.19
3. input_dim = 40 (2 epochs)
sr = 1.17
4. input dim = 100 (1 epoch)
sr = 1.10